# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

In [3]:
from Scratch import load_calibration_memory

DEVICE = 'ibm_sherbrooke'
OTHER_DATE = '2023-10-30'

all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


# Functions:

In [1]:
import numpy as np
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt

def fit_gmm(IQ_data, scaler):

    data = IQ_data.flatten()
    combined_data = np.column_stack((data.real, data.imag))
    combined_data = scaler.transform(combined_data)

    n_components_range = range(1, 4)  

    lowest_bic = np.infty
    best_gmm = None

    for n_components in n_components_range:

        weights_init = [1]

        # Distribute the remaining weight equally among other components
        if n_components > 1:
            # Initialize weights
            weights_init = np.zeros(n_components)
            primary_weight = 0.95  # Weight for the first component
            weights_init = np.zeros(n_components)
            weights_init[0] = primary_weight
            remaining_weight = (1 - primary_weight) / (n_components-1)
            for i in range(1, n_components):
                weights_init[i] = remaining_weight

        if n_components == 2:
            means_init = np.array([[-1e8, 0], [1e8, 0]])
            # means_init = None
        else:
            means_init = None

        # Fit a Gaussian mixture with EM
        gmm = GaussianMixture(n_components=n_components, init_params='random',
                            covariance_type='tied', random_state=0, weights_init=weights_init, means_init=means_init)
        gmm.fit(combined_data)
        
        # Calculate BIC
        bic = gmm.bic(combined_data)
        
        # Check if this is the lowest BIC so far
        if bic < lowest_bic and np.all([weight < 0.1 or weight > 0.9 for weight in gmm.weights_]):
            lowest_bic = bic
            best_gmm = gmm
    
    return best_gmm

def draw_ellipse(mean, cov, n_std=1, ax=None, **kwargs):
    """Draw an ellipse with a given mean and covariance"""
    ax = ax or plt.gca()

    # Eigenvalues and eigenvectors of the covariance matrix
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]

    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * n_std * np.sqrt(vals)
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)

    ax.add_patch(ellipse)


def plot_gmm(gmm, data_2d, n_std=1):
    plt.scatter(data_2d[:, 0], data_2d[:, 1], alpha=0.5)

    # Plot each Gaussian component
    for mean, covar in zip(gmm.means_, gmm.covariances_):
        draw_ellipse(mean, covar, n_std=n_std, alpha=1, edgecolor='red', facecolor='none')

    plt.title("Data and Fitted Gaussian Components")
    plt.xlabel("Real Part")
    plt.ylabel("Imaginary Part")
    plt.show()

# Scale and fit

In [5]:
from soft_info import get_KDEs

kde_dict, scaler_dict = get_KDEs(provider, tobecalib_backend=DEVICE, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


In [40]:
gmm_dict = {}
for qubit in all_memories.keys():
    gmm_dict[qubit] = {}
    scaler = scaler_dict[qubit]
    for mmr in all_memories[qubit].keys():
        data = all_memories[qubit][mmr]
        gmm_dict[qubit][mmr] = fit_gmm(data, scaler)

# takes 3s

# Generate grid

In [21]:
import cpp_soft_info
from Scratch import process_scaler_dict

num_std_dev = 2
num_points = 300

grid_dict = {}

grid_range_real = np.linspace(-num_std_dev, num_std_dev, num_points)
grid_range_imag = np.linspace(-num_std_dev, num_std_dev, num_points)
grid_x, grid_y = np.meshgrid(grid_range_real, grid_range_imag)
grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

for qubit_idx, gmms in gmm_dict.items():
    gmm_0 = gmms['mmr_0']
    gmm_1 = gmms['mmr_1']
    
    grid_density_0 = gmms['mmr_0'].score_samples(grid_points).reshape(grid_x.shape)
    grid_density_1 = gmms['mmr_1'].score_samples(grid_points).reshape(grid_x.shape)

    grid_dict[qubit_idx] = cpp_soft_info.GridData(grid_x, grid_y, grid_density_0, grid_density_1)


# Scaler dict 
processed_scaler_dict = process_scaler_dict(scaler_dict)

# takes 3s

# Decoding